In [1]:
import tensorflow as tf
modelo = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 2264389 (8.64 MB)
Trainable params: 6405 (25.02 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [2]:
from PIL import Image
import requests
from io import BytesIO
import cv2
import numpy as np

def categorizar(url):
  respuesta = requests.get(url)
  img = Image.open(BytesIO(respuesta.content))
  img = np.array(img).astype(float)/255

  img = cv2.resize(img, (224,224))
  prediccion = modelo.predict(img.reshape(-1, 224, 224, 3)) 
  for x in prediccion[0]:
        x = round(x,5)
        print(str(x))
  return np.argmax(prediccion[0], axis=-1)

def categorizar2(path):
    img = Image.open(path)
    img = np.array(img).astype(float)/255
    img = cv2.resize(img, (224,224))
    prediccion = modelo.predict(img.reshape(-1, 224, 224, 3))
    for x in prediccion[0]:
        x = '{:0.3e}'.format(x)
        print(str(x))
    return np.argmax(prediccion[0], axis=-1)

def mostrarCategoria(nombre,window_name):
    print(nombre)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(image, nombre, (10,450), font, 3, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow(window_name,image)
def mostrarImagenCategoria(path):
    imagen = PhotoImage(file = path)
    tacho.config(image=imagen)
    tacho.image = imagen
    


In [3]:
import os


datos = 'capturas'
if not os.path.exists(datos):
    print('Carpeta creada: ',datos)
    os.makedirs(datos)

In [6]:

# Importamos librerias
from tkinter import *
from PIL import Image, ImageTk
import cv2
import imutils
import numpy as np

global lblVideo , cap
global count

count = 0
frame = None

def boton():
    global count,frame 
    cv2.imwrite(datos+'/objeto_{}.jpg'.format(count),frame) ##Se guarda captura en carpeta capturas
    print('Imagen guardada:'+'/objeto_{}.jpg'.format(count))
    path = datos+'/objeto_{}.jpg'.format(count) ## ruta de la captura
    resultado = categorizar2(path) ## devuelve la predicción
    print (resultado) ## imprimo resultado
    if resultado == 0:
        mostrarCategoria("./Tkinter/papel.png")
    elif resultado == 1:
        mostrarCategoria("./Tkinter/vidrio.png")
    elif resultado == 2:
        mostrarImagenCategoria("./Tkinter/metal.png")
    elif resultado == 3:
        mostrarImagenCategoria("./Tkinter/papel.png")
    elif resultado == 4:
        mostrarImagenCategoria("./Tkinter/plastico.png")
    count+=1 ##Se incrementa el valor de contador que se utiliza para no sobreescrivir las imagenes


def visualizar():
    if cap is not None:
        global frame
        ret, frame = cap.read(0)
        if ret == True:
            frame = imutils.resize(frame, width=640)
            # Convertimos el video
            frame_show =cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            im = Image.fromarray(frame_show)
            img = ImageTk.PhotoImage(image=im)

            # Mostramos en el GUI
            lblVideo.configure(image=img)
            lblVideo.image = img
            lblVideo.after(10, visualizar)
        else:
            lblVideo.image = ""
            cap.release()

            
#Variables globales
global tacho #variable global para mostrar el tacho segun el tipo de residuo 

#Parametros iniciales
pantalla = Tk()
pantalla.title("CLASIFICADOR AUTOMATICA DE RESIDUOS")
pantalla.geometry("1280x720")

#fondo
background = PhotoImage(file = "./Tkinter/SortingWaste.png")
background_label = Label(image=background, text="fondo")
background_label.place(x=0, y=0, relwidth=1, relheight=1)

#btn
imagebtn1 = PhotoImage(file = "./Tkinter/Predecir.png")
btn1 = Button(pantalla, text="RGB", image= imagebtn1, height="40", width="200", command=boton)
btn1.place(x = 980, y = 600)

# Lavel de video
lblVideo = Label(pantalla)
lblVideo.place(x = 350, y = 150)

#webcam
cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
cap.set(3, 640)
cap.set(4, 480)

##tacho segun tipo de residuo
tacho = Label(pantalla) #configuro de forma generica para luego actualizar en funcion con el metodo config
tacho.place(x=75, y=260)


visualizar()
pantalla.mainloop()

Imagen guardada:/objeto_0.jpg
1/1 [==============================] - 0s 40ms/step
4.593e-03
1.370e-02
4.466e-02
7.539e-01
1.832e-01
3
